In [ ]:
import random
import pandas as pd 
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
import matplotlib.pyplot as plt
from PIL import Image
import os
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications import VGG16
import tensorflow.keras.applications as tfa
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow import keras
tf.random.set_seed(42)

import wandb
from wandb.keras import WandbCallback
tf.keras.backend.clear_session()
# Set the random seeds
os.environ['TF_CUDNN_DETERMINISTIC'] = '1' 
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

In [ ]:
from keras import backend as K

In [ ]:
dupicate_removed = pd.read_csv('../input/duplicate-removed/duplicate_removed_train.csv')

In [ ]:
def relu_advanced(x):
    return K.relu(x, max_value=1.)

In [ ]:
wandb.login(key='620527a80f5b194ce6ba9498879a2ebe65db428d')
#620527a80f5b194ce6ba9498879a2ebe65db428d

In [ ]:
wandb.init(reinit=True)

1. add image sizes to config 
2. and data augmentation

In [ ]:
#model_logit.compile(loss=bce_with_logits, optimizer="RMSprop")

In [ ]:
#meta = pd.read_csv('/kaggle/input/petfinder-pawpularity-score/train.csv')
meta = pd.read_csv('../input/duplicate-removed/duplicate_removed_train.csv')

In [ ]:
def append_ext(fn):
    return fn+".jpg"

In [ ]:
labels = meta[['Id','Pawpularity']]
labels['Pawpularity_norm'] = labels['Pawpularity']/100
labels["Id"] = labels["Id"].apply(append_ext)
labels.to_csv('labels.csv',encoding='utf-8', index=False)

In [ ]:
def load_train(path):
        # path /kaggle/input/petfinder-pawpularity-score/
    labels = pd.read_csv('./labels.csv')
       
    train_datagen = ImageDataGenerator(validation_split = 0.25, 
                                   #rescale = 1/255.
                                   #vertical_flip = True, 
                                   horizontal_flip = True
                                   #rotation_range = 45,
                                   #width_shift_range=0.05,
                                   #height_shift_range=0.05
                                      )
    
    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory= path +'train/',
        x_col='Id',
        y_col='Pawpularity_norm',
        target_size=(224, 224),
        batch_size=64,
        class_mode='raw',
        subset='training',
        seed=42)

    return train_gen_flow

In [ ]:
def load_test(path):
    labels = pd.read_csv('./labels.csv')
    test_datagen = ImageDataGenerator(
        validation_split=0.25,
        horizontal_flip = True
        #rescale=1./255
    )
    
# flow_from_dataframe -> Takes the dataframe and the path to a directory + generates batches.
# The generated batches contain augmented/normalized data.
    test_gen_flow = test_datagen.flow_from_dataframe(
        dataframe = labels,
        directory = path + 'train/',
        x_col = 'Id',
        y_col = 'Pawpularity_norm',
        target_size = (224, 224),
        batch_size = 64,
        class_mode = 'raw',
        subset = 'validation',
        seed = 42)

    return test_gen_flow

In [ ]:
path = '/kaggle/input/petfinder-pawpularity-score/'
train_gen_flow = load_train(path)

In [ ]:
path = '/kaggle/input/petfinder-pawpularity-score/'
test_gen_flow = load_test(path)

In [ ]:
def create_model(input_shape, model_name, fine_tune_from, head, opt, loss, activation, lr):

    backbone = model_name(weights='imagenet', 
                        input_shape=input_shape,
                        include_top=False)
    
    inputs = keras.Input(shape=input_shape)
    x = inputs
    x = tf.keras.applications.resnet50.preprocess_input(x)
    x = backbone(x, training=True)

    set_trainable = False
    for layer in backbone.layers:
        if fine_tune_from in layer.name :
            set_trainable = True
        if set_trainable:
            layer.trainable = True
        else:
            layer.trainable = False

    if head == 'GPA_sigmoid':
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        output = keras.layers.Dense(1, activation='sigmoid')(x)
    
    elif head == 'GPA_logits':
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        output = keras.layers.Dense(1)(x)
        
    elif head == 'Flatten_FC_sigmoid_DO':
    
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(0.2)(x)
        output = keras.layers.Dense(1,activation='sigmoid')(x)
        
    elif head == 'Flatten_FC_relu_DO':
    
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(0.2)(x)
        output = keras.layers.Dense(1)(x)
    
    elif head == 'GAP_FC_relu_DO':
    
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(0.2)(x)
        output = keras.layers.Dense(1)(x)
    
    elif head == 'MP_relu128_DO_BN_relu64':
    
        x =  tf.keras.layers.MaxPooling2D()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(0.2)(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        output = keras.layers.Dense(1)(x)
        
    elif head == 'GAP_relu128_DO_BN_relu64':
    
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(0.4)(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        output = keras.layers.Dense(1)(x)
    
    elif head == 'GAP_relu128_DO_BN_relu64_DO':
    
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(0.2)(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(64, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(0.2)(x)
        output = keras.layers.Dense(1)(x)
        
    elif head == 'GAP_FC_relu_DO_BN':
    
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(0.2)(x)
        x = keras.layers.BatchNormalization()(x)
        output = keras.layers.Dense(1)(x)

    elif head == 'GAP_FC_relu512_DO_BN_relu128_DO_BN':
    
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(0.3)(x)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.Dense(128, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(0.2)(x)
        x = keras.layers.BatchNormalization()(x)
        output = keras.layers.Dense(1)(x)
        
    elif head == 'GAP_relu512_DO_BN_sigmoid':
        
        x =  tf.keras.layers.GlobalAveragePooling2D()(x)
        x = keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        x = keras.layers.Dropout(0.2)(x)
        x = keras.layers.BatchNormalization()(x)
        output = keras.layers.Dense(1,activation='sigmoid')(x)
        
    
    elif head =='MP_DO_FL_relu512':
        x =  tf.keras.layers.MaxPooling2D()(x)
        x = keras.layers.Dropout(0.2)(x)
        x = keras.layers.Flatten()(x)
        x = keras.layers.Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.HeNormal())(x)
        output = keras.layers.Dense(1)(x)
        
    model = tf.keras.Model(inputs=inputs, outputs=output)
    
    if loss == 'mse':

        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                    loss=tf.keras.losses.MeanSquaredError(),
                    metrics=[tf.keras.metrics.RootMeanSquaredError()])
    elif loss == 'bce_logits':
        
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
        
    elif loss == 'bce_no_logits':
        model.compile(
                  optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])
    
    return model

1. add batchNormalization after DO (Dropout is meant to block information from certain neurons completely to make sure the neurons do not co-adapt. So, the batch normalization has to be after dropout otherwise you are passing information through normalization statistics. https://stackoverflow.com/questions/39691902/ordering-of-batch-normalization-and-dropout)
2. change number of outputs 128-64 
3. change flatten to GAP
4. change relu to Swish & SiLU SELU or ELU, use LeCun initialization.
4. Adam to SGD and made more epochs

In [ ]:
config = dict (
    architecture = "CNN",
    dataset_id = "Petfinder_wo_duplicates",
    infra = "Kaggle",
    pictures_normalization = 'imageNet',
    input_shape = (224, 224, 3),
    learning_rate = 0.01,
    lr_decay = True,
    pre_trained_model = ResNet50,
    tune_from = "conv3",
    head = "GAP_relu128_DO_BN_relu64",
    dropout = 0.4,
    activation = "relu",
    optimizer = Adam,
    
    epochs=1,
    batch_size=64,
    loss_function="bce_no_logits",
    metric="rmse",
    
)

wandb.init(
  project="Petfinder",
  entity="makcfd",
  notes="W&B_init",
  #tags=["baseline", "paper1"],
  config=config,
)

In [ ]:
optimizer = config['optimizer']
pre_trained_model = config['pre_trained_model']
fine_tune_from = config['tune_from']
head = config['head']
activation = config['head']
input_shape = config['input_shape']
loss = config['loss_function']

lr = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=config['learning_rate'],
    decay_steps=100, decay_rate=0.96,
    staircase=True)

model_to_check = create_model(input_shape, 
                              pre_trained_model, 
                              fine_tune_from, 
                              head, 
                              optimizer,
                              loss,
                              activation, 
                              lr)

In [ ]:
model_to_check.summary()

In [ ]:
def train_model(model, train_data, test_data, batch_size = None, epochs = config['epochs'],
                steps_per_epoch = None, validation_steps = None):

    if steps_per_epoch is None:
        steps_per_epoch = len(train_data)
    if validation_steps is None:
        validation_steps = len(test_data)

    history = model.fit(train_data, 
              validation_data = test_data,
              batch_size = batch_size, 
              epochs = epochs,
              steps_per_epoch = steps_per_epoch,
              validation_steps = validation_steps,
              verbose = 2,
              callbacks=[WandbCallback()])

    return history

In [ ]:
tf.keras.backend.clear_session()
history = train_model(model_to_check, train_gen_flow, test_gen_flow, batch_size = 64)

In [ ]:
wandb.finish()